# Cross-Country Quantile Forecasting with Factor Neural Networks

This notebook demonstrates a complete end-to-end workflow for multi-horizon quantile forecasting across countries using Factor Neural Networks (FNN) and Linear Quantile Regression (LQR).

## Overview

The workflow includes:
1. **Data Loading & Exploration**: Load country time series data and explore patterns
2. **Data Preprocessing**: Handle missing values, scaling, and create lagged features
3. **Model Training**: Train both Factor Neural Networks and Linear Quantile Regression
4. **Model Evaluation**: Compare models using quantile loss and Diebold-Mariano tests
5. **Visualization**: Create forecast plots, calibration diagrams, and factor analysis

## Key Features

- **Multi-country analysis** with cross-validation across countries
- **Multi-horizon forecasting** (1, 4, 8 periods ahead)
- **Quantile regression** for uncertainty quantification
- **Factor interpretation** from neural network last layers
- **Statistical testing** for model comparison

## 1. Import Required Libraries

Import essential libraries for data processing, modeling, and visualization.

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import yaml
from datetime import datetime


# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Project imports (adjust paths as needed)
import sys
sys.path.append('..')

from src.utils import (
    load_country_data, 
    load_config,
    set_seeds,
    handle_missing_values,
    scale_features,
    create_lagged_features,
    create_forecast_targets,
    create_time_split
)

# Import high-level API classes
from src.ensemble_nn_api import EnsembleNNAPI  # High-level API from ensemble_nn_api.py
from src.lqr_api import LQRModel        # High-level API from lqr_api.py

# Alternative: Low-level API (from the packages)

from src.metrics import (
    pinball_loss,
    compute_quantile_losses,
    create_loss_summary_table,
    dm_test_by_groups,
    multiple_testing_correction
)

from src.evaluation import (
    create_model_comparison_table,
    plot_training_curves,
    plot_forecast_paths,
    plot_loss_comparison,
    plot_calibration,
    create_evaluation_dashboard
)

import torch 
torch.set_num_threads(4) 


print("Libraries imported successfully!")
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

# print(f"LQRModel class: {LQRModel}")

: 

In [ ]:
# Enable auto-reload for development
# This will automatically reload modules when you change the source code
%load_ext autoreload
%autoreload 2

# Optional: For more verbose reloading info
# %autoreload 1  # Only reload modules imported with %aimport
# %autoreload 0  # Disable autoreload

print("Auto-reload enabled: Changes to source files will be automatically reloaded!")

## 2. Configuration and Setup

Load configuration and set up the experiment parameters.

In [ ]:
# Load configuration
config_path = "../configs/experiment_big_data.yaml"

try:
    config = load_config(config_path)
    print("Configuration loaded successfully!")
    print("\nKey parameters:")
    print(f"Target variable: {config['data']['target']}")
    print(f"Quantiles: {config['data']['quantiles']}")
    print(f"Horizons: {config['data']['horizons']}")
    print(f"Lags: {config['data']['lags']}")
    print(f"Validation size: {config['splits']['validation_size']}")
except FileNotFoundError:
    print(f"Configuration file not found at {config_path}")
    print("Using default configuration...")
    
    # Default configuration
    config = {
        'seed': 123,
        'data': {
            'target': 'GDP',
            'required_columns': ['TIME', 'CISS', 'INFL_OECD', 'GDP', 'CREDIT_TO_GDP'],
            'quantiles': [0.05, 0.95],
            'horizons': [1],
            'lags': [1],
            'missing': 'forward_fill_then_mean',
            'scale': 'per_country',
            'scale_target': False
        },
        'splits': {
            'validation_size': 0.2,
            'train_start': '1990-12-31',
            'test_cutoff': '2018-12-31',
            'min_train_points': 60
        },
        'ensemble_nn': {
            'units_per_layer': [8, 4],
            'activation': 'relu',
            'optimizer': 'adam',
            'learning_rate': 1e-3,
            'epochs': 50,  # Reduced for demo
            'batch_size': 64,
            'patience': 10,
            'parallel_models': 1,  # Single model for demo
            'device': 'auto'
        },
        'lqr': {
            'alphas': [0.0, 0.1, 1.0, 10.0],
            'solver': 'huberized'
        }
    }

# Set random seed
set_seeds(config['seed'])
print(f"\nRandom seed set to: {config['seed']}")

## 3. Load and Explore Dataset

Load country time series data and perform exploratory data analysis.

In [ ]:
# Load country data
data_path = "../processed_per_country"

try:
    # Load all country CSV files
    country_data = load_country_data(
        data_path=data_path,
        required_columns=config['data']['required_columns']
    )
    
    print(f"Successfully loaded data for {len(country_data)} countries:")

    for country, df in country_data.items():

        print(f"  {country}: {len(df)} observations, {df['TIME'].min()} to {df['TIME'].max()}")
    
    # Display first few rows of a sample country
    sample_country = list(country_data.keys())[0]
    print(f"\nSample data from {sample_country}:")
    display(country_data[sample_country].head())
    
    # Summary statistics
    print(f"\nSummary statistics for {sample_country}:")
    display(country_data[sample_country].describe())
    
except FileNotFoundError:
    print(f"Data directory not found at {data_path}")
    print("Creating synthetic data for demonstration...")
    
    # Create synthetic data for demonstration
    countries = ['USA', 'DEU', 'FRA', 'GBR']
    start_date = '2000-01-01'
    end_date = '2023-12-31'
    freq = 'Q'  # Quarterly data
    
    country_data = {}
    np.random.seed(config['seed'])
    
    for country in countries:
        dates = pd.date_range(start_date, end_date, freq=freq)
        n_obs = len(dates)
        
        # Generate synthetic time series with trends and cycles
        t = np.arange(n_obs)
        trend = 0.01 * t + np.random.normal(0, 0.1, n_obs).cumsum()
        
        df = pd.DataFrame({
            'TIME': dates,
            'GDP': 100 + trend + np.random.normal(0, 2, n_obs),
            'CISS': np.random.beta(2, 5, n_obs),
            'INFL_OECD': np.random.normal(2, 1, n_obs),
            'CREDIT_TO_GDP': 150 + 10 * np.sin(t / 20) + np.random.normal(0, 5, n_obs)
        })
        
        country_data[country] = df
    
    print(f"Created synthetic data for {len(country_data)} countries")
    sample_country = countries[0]
    print(f"\nSample synthetic data from {sample_country}:")
    display(country_data[sample_country].head())

# manually put the columns names in the config
config['data']['required_columns'] = list(country_data[sample_country].columns)
print(f"\nUpdated required columns in config: {config['data']['required_columns']}")

In [ ]:
# Visualize the data
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
axes = axes.flatten()

variables = config['data']['required_columns'][1:]  # Exclude 'TIME'
colors = plt.cm.Set1(np.linspace(0, 1, len(country_data)))

for i, var in enumerate(variables[:6]):
    ax = axes[i]
    
    for j, (country, df) in enumerate(country_data.items()):
        ax.plot(df['TIME'], df[var], label=country, color=colors[j], linewidth=2)
    
    ax.set_title(f'{var} Across Countries')
    ax.set_xlabel('Time')
    ax.set_ylabel(var)
    #ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Correlation analysis
print("\nCorrelation analysis:")
for country, df in list(country_data.items())[:2]:  # Show first 2 countries
    print(f"\n{country} - Correlation matrix:")
    corr_matrix = df[variables].corr()
    display(corr_matrix.round(3))

## 4. Data Preprocessing

Handle missing values, scaling, and create time splits.

In [ ]:
# 1. Handle missing values
print("Handling missing values...")
processed_data = handle_missing_values(
    country_data, 
    policy=config['data']['missing']
)

# Check for missing values
print("\nMissing values after processing:")
for country, df in processed_data.items():
    missing_count = df.isnull().sum().sum()
    print(f"{country}: {missing_count} missing values")

# 2. Create time splits
print("\nCreating time splits...")
train_data, test_data, dropped_countries = create_time_split(
    processed_data,
    train_start=config['splits']['train_start'],
    test_cutoff=config['splits']['test_cutoff'],
    min_train_points=config['splits']['min_train_points']
)

print(f"Train/test split created:")
print(f"  Training countries: {len(train_data)}")
print(f"  Test countries: {len(test_data)}")
print(f"  Dropped countries: {dropped_countries}")

# Show split sizes
for country in list(train_data.keys())[:3]:  # First 3 countries
    train_size = len(train_data[country])
    test_size = len(test_data[country])
    print(f"  {country}: {train_size} train, {test_size} test")

# 3. Scale features (on training data)
print("\nScaling features...")
scaled_train, scaled_test, scalers = scale_features(
    train_data=train_data,
    test_data=test_data,
    policy=config['data']['scale'],
    target_col=config['data']['target'],
    scale_target=config['data']['scale_target'],
    trimming=config['data']['trimming'],  # Optional trimming parameter
)

print(f"Feature scaling completed using '{config['data']['scale']}' policy")

## 5. Feature Engineering

Create lagged features and forecast targets for multi-horizon prediction.

In [ ]:
# 1. Create lagged features
print("Creating lagged features...")
lagged_train = create_lagged_features(
    scaled_train, 
    lags=config['data']['lags']
)

lagged_test = create_lagged_features(
    scaled_test, 
    lags=config['data']['lags']
)

# Show example of lagged features
sample_country = list(lagged_train.keys())[0]
print(f"\nLagged features for {sample_country}:")
print(f"Original columns: {list(scaled_train[sample_country].columns)}")
print(f"With lags: {list(lagged_train[sample_country].columns)}")
print(f"Data shape: {lagged_train[sample_country].shape}")

# 2. Create forecast targets
print("\nCreating forecast targets...")
target_train = create_forecast_targets(
    lagged_train,
    target_col=config['data']['target'],
    horizons=config['data']['horizons']
)

target_test = create_forecast_targets(
    lagged_test,
    target_col=config['data']['target'],
    horizons=config['data']['horizons']
)

# Show target structure
target_cols = [f"{config['data']['target']}_h{h}" for h in config['data']['horizons']]
print(f"Target columns: {target_cols}")

for country in list(target_train.keys())[:2]:
    print(f"{country} training shape: {target_train[country].shape}")
    print(f"{country} test shape: {target_test[country].shape}")

# Display sample of processed data
print(f"\nProcessed data sample from {sample_country}:")
display(target_train[sample_country][['TIME'] + target_cols].head())

## 6. Model Training

Train both Factor Neural Networks and Linear Quantile Regression with cross-validation.

In [ ]:
def create_dummy_variables(data_list, time_col="TIME", add_country_dummies=False):
    # for each country in the data list add a country encoding variable 
    num_of_countries = len(data_list)
    if not add_country_dummies:
        return data_list
    
    new_data_list = []

    for i,  df in enumerate(data_list):
        df_copy = df.copy()
        for j in range(num_of_countries):
            dummy_col = f'dummy-{j}'
            if i != j:
                df_copy[dummy_col] = 0
            else:
                df_copy[dummy_col] = 1
        new_data_list.append(df_copy)
        
    # for i, df in enumerate(data_list):
    #     df_copy = df.copy()
    #     # Use simple integer encoding instead of one-hot dummies
    #     df_copy['country_id'] = i
    #     new_data_list.append(df_copy)


    
    return new_data_list


In [ ]:
# Prepare data for models
# Convert country dictionaries to lists for model APIs
import traceback
from src.ensemble_nn_api import EnsembleNNAPI  # High-level API from ensemble_nn_api.py
from src.lqr_api import LQRModel        # High-level API from lqr_api.py

train_data_list = []
test_data_list = []

for country, df in scaled_train.items():
    df_copy = df.copy()
    #df_copy['country'] = country
    train_data_list.append(df_copy)

for country, df in scaled_test.items():
    df_copy = df.copy()
    #df_copy['country'] = country
    test_data_list.append(df_copy)

print(f"Prepared data for {len(train_data_list)} countries")

# Ensure all data types are proper (TIME should be datetime, others numeric)
for i, df in enumerate(train_data_list):
    if 'TIME' in df.columns and df['TIME'].dtype == 'object':
        train_data_list[i]['TIME'] = pd.to_datetime(df['TIME'])
            
for i, df in enumerate(test_data_list):
    if 'TIME' in df.columns and df['TIME'].dtype == 'object':
        test_data_list[i]['TIME'] = pd.to_datetime(df['TIME'])


if config['ensemble_nn']['country_dummies'] and not config['ensemble_nn']['per_country']:
    global_train_data_list = create_dummy_variables(
        train_data_list,
        time_col="TIME",
        add_country_dummies=config['ensemble_nn']['country_dummies']
    )
    
    global_test_data_list = create_dummy_variables(
        test_data_list,
        time_col="TIME",
        add_country_dummies=config['ensemble_nn']['country_dummies']
    )
else:
    global_train_data_list = train_data_list
    global_test_data_list = test_data_list



In [ ]:


# === Train Factor Neural Network ===
print("\n" + "="*50)
print("Training Factor Neural Network")
print("="*50)

# Determine optimal k_folds based on available countries

n_countries = len(train_data_list)

print(f"Available countries: {n_countries}")

country_names = [country for country in list(target_train.keys())]

if config['ensemble_nn']['per_country']:
    fnn_models = {}
    for i, df in enumerate(train_data_list):
       
        print(f"Training Factor NN for {i}...")

        ensemble_nn = EnsembleNNAPI(
            data_list=[df],
            target=config['data']['target'],
            quantiles=config['data']['quantiles'],
            forecast_horizons=config['data']['horizons'],
            units_per_layer=config['ensemble_nn']['units_per_layer'],
            lags=config['data']['lags'],
            activation=config['ensemble_nn']['activation'],
            device=config['ensemble_nn']['device'],
            seed=config['seed'], 
            transform=True,  # Enable data transformation
        )
    
        ensemble_results = ensemble_nn.fit(
            epochs=config['ensemble_nn']['epochs'],
            learning_rate=config['ensemble_nn']['learning_rate'],
            batch_size=config['ensemble_nn']['batch_size'],
            validation_size=config['splits']['validation_size'],  # Use adjusted k_folds
            patience=config['ensemble_nn']['patience'],
            verbose=1,
            optimizer=config['ensemble_nn']['optimizer'],
            parallel_models=config['ensemble_nn']['parallel_models'],
            l2=config['ensemble_nn']['l2_penalty'],
            return_validation_loss=True,
            return_train_loss=True,
            shuffle=True
        )

        print(f"✓ Factor NN for {i} trained successfully!")

        # Store model and results
        fnn_models[i] = {
            'model': ensemble_nn,
            'results': ensemble_results
        }
        
         # Plot training curves if available
        if 'train_losses' in ensemble_results and 'val_losses' in ensemble_results:
            plot_training_curves(
                ensemble_results['train_losses'][min(config['ensemble_nn']['patience']-1,5):],
                ensemble_results['val_losses'][min(config['ensemble_nn']['patience']-1,5):],
                title="Factor NN Training Curves"
            )
            plt.show()
    
    print(f"\nTrained Factor NN models for {len(fnn_models)} countries.")
else:

  
    # Initialize Ensemble NN using high-level API
    ensemble_nn = EnsembleNNAPI(
        data_list=global_train_data_list,
        target=config['data']['target'],
        quantiles=config['data']['quantiles'],
        forecast_horizons=config['data']['horizons'],
        units_per_layer=config['ensemble_nn']['units_per_layer'],
        lags=config['data']['lags'],
        activation=config['ensemble_nn']['activation'],
        device=config['ensemble_nn']['device'],
        seed=config['seed'], 
        transform=True,  # Enable data transformation
        prefit_AR=True,  # Enable pre-fitting of AR models
        country_ids=country_names,  # Use country names
        time_col="TIME",
        verbose=1
    )
    
    print(f"✓ Ensemble NN initialized successfully!")
    print(f"  Input dimension: {ensemble_nn.input_dim}")
    print(f"  Target quantiles: {ensemble_nn.quantiles}")
    print(f"  Forecast horizons: {ensemble_nn.forecast_horizons}")

    # Train with cross-validation
    print("\nStarting training with cross-validation...")
    ensemble_results = ensemble_nn.fit(
        epochs=config['ensemble_nn']['epochs'],
        learning_rate=config['ensemble_nn']['learning_rate'],
        batch_size=config['ensemble_nn']['batch_size'],
        validation_size=config['splits']['validation_size'],  # Use adjusted k_folds
        patience=config['ensemble_nn']['patience'],
        verbose=1,
        optimizer=config['ensemble_nn']['optimizer'],
        parallel_models=config['ensemble_nn']['parallel_models'],
        l2=config['ensemble_nn']['l2_penalty'],
        return_validation_loss=True,
        return_train_loss=True,
        shuffle=True
    )

    print("\n✓ Ensemble NN training completed successfully!")
    print(f"  Results keys: {list(ensemble_results.keys())}")
    print(f"  Number of parameters: {ensemble_results.get('n_parameters', 'Unknown')}")
    print(f"  Final validation loss: {ensemble_results.get('final_val_loss', 'Unknown')}")

    # Plot training curves if available
    if 'train_losses' in ensemble_results and 'val_losses' in ensemble_results:
        plot_training_curves(
            ensemble_results['train_losses'],#[min(config['ensemble_nn']['patience']-1,5):],
            ensemble_results['val_losses'],#[min(config['ensemble_nn']['patience']-1,5):],
            title="Ensemble NN Training Curves"
        )
        plt.show()
    
    # Mark as successfully trained
    fnn_training_successful = True
        
   
    # === Status Summary ===
    print("\n" + "="*50)
    print("TRAINING STATUS SUMMARY")
    print("="*50)
    print(f"✓ Data preprocessing: SUCCESSFUL")
    print(f"✓ Feature engineering: SUCCESSFUL") 
    print(f"✓ Dataset creation: SUCCESSFUL")
    print(f"✓ Data type handling: SUCCESSFUL (fixed object dtype issue)")
    print(f"{'✓' if fnn_training_successful else '🚨'} Factor NN training: {'SUCCESSFUL' if fnn_training_successful else 'FAILED (cross-validation issue)'}")

    if not fnn_training_successful:
        print(f"\nNote: The core infrastructure is working correctly.")
        print(f"The training failure is due to cross-validation configuration,")
        print(f"not fundamental data type or model implementation issues.")


In [ ]:
# === Train Linear Quantile Regression ===
print("\n" + "="*50)
print("Training Linear Quantile Regression")
print("="*50)

try:
    # Initialize LQR using high-level API
    lqr_model = LQRModel(
        data_list=train_data_list,
        target=config['data']['target'],
        quantiles=config['data']['quantiles'],
        forecast_horizons=config['data']['horizons'],
        lags=config['data']['lags'],
        alpha=1.0,  # Will be cross-validated
        fit_intercept=True,
        solver=config['lqr']['solver'],
        seed=config['seed']
    )
    
    print(f"LQR initialized")
    
    # Cross-validate regularization parameter
    print("Cross-validating regularization parameter...")
    best_alpha = lqr_model.validate_alpha(
        alphas=config['lqr']['alphas'],
        validation_size=config['splits']['validation_size']
    )
    
    print(f"Best alpha: {best_alpha}")
    
    # Fit final model
    print("Fitting final model...")
    lqr_coefficients = lqr_model.fit()
    
    print("LQR training completed!")

    
except Exception as e:
    print(f"LQR training failed: {e}")
    print(f"Error type: {type(e).__name__}")
    print(f"Error location:")
    traceback.print_exc()
    print("This might be due to missing dependencies. Continuing with synthetic model results...")
    lqr_model = None
    lqr_coefficients = None

print("\nModel training phase completed!")

In [ ]:
### lqr per country
print("\n" + "="*50)
print("LQR Coefficients by Country")
print("="*50)
lqr_models = {}

for i, data in enumerate(train_data_list):
    # Initialize LQR using high-level API
    if i == 0:
        print(data.head())
    lqr_model = LQRModel(
        data_list=[data],
        target=config['data']['target'],
        quantiles=config['data']['quantiles'],
        forecast_horizons=config['data']['horizons'],
        lags=config['data']['lags'],
        alpha=1.0,  # Will be cross-validated
        fit_intercept=True,
        solver=config['lqr']['solver'],
        seed=config['seed']
    )

    
    # Cross-validate regularization parameter
    # Cross-validate regularization parameter
    best_alpha = lqr_model.k_fold_validation(alphas = config['lqr']['alphas'],
        n_splits=10
    )
    
    print(f"Best alpha for country {countries[i]}: {best_alpha}")
    
    # Fit final model
    lqr_coefficients = lqr_model.fit()
    

    lqr_models[i] = {
        'model': lqr_model,
        'coefficients': lqr_coefficients
    }





    

In [ ]:
### lqr per country
print("\n" + "="*50)
print("LQR Coefficients by Country")
print("="*50)
lqr_ar_models = {}

for i, data in enumerate(train_data_list):
    # Initialize LQR using high-level API
    if i == 0:
        print(data.head())
    
    # only use the target column and the time column
    data_temp = data[['TIME', config['data']['target']]].copy()

    lqr_model = LQRModel(
        data_list=[data_temp],
        target=config['data']['target'],
        quantiles=config['data']['quantiles'],
        forecast_horizons=config['data']['horizons'],
        lags=config['data']['lags'],
        alpha=0.0,  # Will be cross-validated
        fit_intercept=True,
        solver=config['lqr']['solver'],
        seed=config['seed']
    )

    
    # Cross-validate regularization parameter
    # Cross-validate regularization parameter
    # best_alpha = lqr_model.k_fold_validation(alphas = config['lqr']['alphas'],
    #     n_splits=10
    # )
    
    #print(f"Best alpha for country {i}: {best_alpha}")
    
    # Fit final model
    lqr_coefficients = lqr_model.fit()
    

    lqr_ar_models[i] = {
        'model': lqr_model,
        'coefficients': lqr_coefficients
    }



In [ ]:
# def normalize(upb, unb, data):
#     # normalize per column 
#     return (data - data.min()) / (data.max() - data.min()) * (upb - unb) + unb


# def add_random_fourier_features(data, target, n_features=10, seed=42):
    
#     np.random.seed(seed)
    
#     # Generate random weights and biases
#     weights = np.random.randn(n_features, data.shape[1] - 1)  # Exclude TIME column
#     biases = np.random.uniform(0, 2 * np.pi, n_features)

#     target_data = data[target]
#     # drop target from data 

    
#     new_data = pd.DataFrame()

#     # Create new features - both sin and cos for each random projection
#     for i in range(n_features):
#         projection = np.dot(data.iloc[:, 1:].values , weights[i]) + biases[i]
#         # Add both sin and cos features
#         new_data[f'rff_cos_{i}'] = np.cos(projection)
#         new_data[f'rff_sin_{i}'] = np.sin(projection)

#     new_data[target] = target_data

#     return new_data

In [ ]:
# ### lqr per country
# print("\n" + "="*50)
# print("LQR Coefficients by Country")
# print("="*50)
# rff_lqr_models = {}

# for i, data in enumerate(train_data_list):

#     # add random fourier features to data 
#     data = add_random_fourier_features(data, target=config['data']['target'], n_features=config['lqr']['rff_features'], seed=config['seed'])
    
#     # print dimensions of data
#     print(f"Data shape for country {i}: {data.shape}")
#     # Initialize LQR using high-level API
#     lqr_model = LQRModel(
#         data_list=[data],
#         target=config['data']['target'],
#         quantiles=config['data']['quantiles'],
#         forecast_horizons=config['data']['horizons'],
#         lags=config['data']['lags'],
#         alpha=1.0,  # Will be cross-validated
#         fit_intercept=True,
#         solver=config['lqr']['solver'],
#         seed=config['seed']
#     )

    
#     # Cross-validate regularization parameter
#     best_alpha = lqr_model.k_fold_validation(alphas = config['lqr']['alphas'],
#         n_splits=10
#     )
    
#     print(f"Best alpha for country {i}: {best_alpha}"   )
#     # Fit final model
#     lqr_coefficients = lqr_model.fit()
    

#     rff_lqr_models[i] = {
#         'model': lqr_model,
#         'coefficients': lqr_coefficients
#     }



In [ ]:
lqr_per_country_predictions, lqr_per_country_targets = {}, {}
country_names = [country for country in list(target_train.keys())]
for idx, test_data in enumerate(test_data_list):
    model = lqr_models[idx]['model']
    predictions, targets = model.predict([test_data])
    lqr_per_country_predictions[country_names[idx]] = predictions
    lqr_per_country_targets[country_names[idx]] = targets

lqr_ar_per_country_predictions, lqr_ar_per_country_targets = {}, {}

for idx, test_data in enumerate(test_data_list):
    model = lqr_ar_models[idx]['model']
    predictions, targets = model.predict([test_data[['TIME', config['data']['target']]]])
    lqr_ar_per_country_predictions[country_names[idx]] = predictions
    lqr_ar_per_country_targets[country_names[idx]] = targets

print(f"Number of countries with LQR predictions: {len(lqr_per_country_predictions)}")
print(f"Number of countries with LQR AR predictions: {len(lqr_ar_per_country_predictions)}")

fnn_per_country_predictions, fnn_per_country_targets = {}, {}

for idx, test_data in enumerate(global_test_data_list):

    if config['ensemble_nn']['per_country']:
        model = fnn_models[idx]['model']
        predictions, targets = model.predict_per_country(test_data, "COUNTRY_000")
        fnn_per_country_predictions[country_names[idx]] = predictions
        fnn_per_country_targets[country_names[idx]] = targets
    else:
        if ensemble_nn:
            predictions, targets = ensemble_nn.predict_per_country(test_data, country_names[idx])
  
            fnn_per_country_predictions[country_names[idx]] = predictions
            fnn_per_country_targets[country_names[idx]] = targets
        else:
            fnn_per_country_predictions[country_names[idx]] = None
            fnn_per_country_targets[country_names[idx]] = None

print(f"FNN predictions: {len(fnn_per_country_predictions)} countries")
print(f"LQR predictions: {len(lqr_per_country_predictions)} countries")

# print(lqr_per_country_targets["HUN"][:,0])
# print(fnn_per_country_targets["HUN"][:,0])

# rff_per_country_predictions, rff_per_country_targets = {}, {}

# for idx, test_data in enumerate(test_data_list):
#     model = rff_lqr_models[idx]['model']
#     test_data = add_random_fourier_features(test_data, n_features=config['lqr']['rff_features'], seed=config['seed'])
#     predictions, targets = model.predict([test_data])
#     rff_per_country_predictions[country_names[idx]] = predictions
#     rff_per_country_targets[country_names[idx]] = targets


fnn_per_country_predictions = lqr_ar_per_country_predictions
nn_per_country_targets = lqr_ar_per_country_targets
# lqr_per_country_predictions = lqr_ar_per_country_predictions
# lqr_per_country_targets = lqr_ar_per_country_targets


## 6. Model Evaluation

Now let's evaluate our trained FNN model and make predictions.

In [ ]:
fnn_predictions, fnn_targets = ensemble_nn.predict(global_test_data_list) if ensemble_nn else (None, None)
#lqr_predictions, lqr_targets = lqr_model.predict(test_data_list) if lqr_model else (None, None)
#print(lqr_predictions )
# #lqr_predictions, lqr_targets = lqr_per_country_predictions, lqr_per_country_targets

# print(fnn_targets.shape, lqr_targets.shape)

# # === Evaluate Models ===
# print("\n" + "="*50)
# print("MODEL EVALUATION")
# print("="*50)

# # Display prediction shapes for understanding
# if fnn_predictions is not None:
#     print(f"FNN predictions shape: {fnn_predictions.shape}")
#     print(f"  - Dimension 0 (samples): {fnn_predictions.shape[0]} stacked across all countries")
#     print(f"  - Dimension 1 (quantiles): {fnn_predictions.shape[1]} quantiles")
#     print(f"  - Dimension 2 (horizons): {fnn_predictions.shape[2]} horizons")

# if lqr_predictions is not None:
#     print(f"LQR predictions shape: {lqr_predictions.shape}")

# if fnn_targets is not None:
#     print(f"FNN targets shape: {fnn_targets.shape}")
#     print(f"  - Dimension 0 (samples): {fnn_targets.shape[0]} stacked across all countries")
#     print(f"  - Dimension 1 (horizons): {fnn_targets.shape[1]} horizons")

# # Create synthetic predictions for demonstration if models failed
# if fnn_predictions is None and lqr_predictions is None:
#     print("Creating synthetic predictions for demonstration...")
    
#     # Generate synthetic predictions with realistic patterns
#     np.random.seed(config['seed'])
#     n_test_points = sum(len(df) for df in test_data_list)
#     n_quantiles = len(config['data']['quantiles'])
#     n_horizons = len(config['data']['horizons'])
    
#     # Create 3D arrays: (samples, quantiles, horizons)
#     fnn_predictions = np.random.normal(0, 1, (n_test_points, n_quantiles, n_horizons))
#     lqr_predictions = np.random.normal(0, 1.2, (n_test_points, n_quantiles, n_horizons))
    
#     # Create 2D targets: (samples, horizons)
#     fnn_targets = lqr_targets = np.random.normal(0, 1, (n_test_points, n_horizons))
    
#     print("✓ Synthetic predictions created for evaluation demonstration")

# # === 1. Compute Tick Loss (Quantile Loss) ===
# print("\n1. Computing Tick Loss for all quantiles and horizons...")

# from src.metrics import pinball_loss

# tick_losses = {'FNN': {}, 'LQR': {}}

# # Calculate tick losses for each horizon and quantile
# for h, horizon in enumerate(config['data']['horizons']):
#     print(f"\nHorizon {horizon} (index {h}):")
#     tick_losses['FNN'][horizon] = {}
#     tick_losses['LQR'][horizon] = {}
    
#     # Get targets for this horizon
#     # If targets is 2D: (samples, horizons), get column h
#     # If targets is 1D and only one horizon, use as-is
#     if fnn_targets.ndim == 2:
#         actual = fnn_targets[:, h]
#     else:
#         actual = fnn_targets  # Single horizon case
    
#     for q, quantile in enumerate(config['data']['quantiles']):
#         # FNN tick loss: get predictions for quantile q, horizon h
#         if fnn_predictions is not None:
#             fnn_pred = fnn_predictions[:, q, h]  # (samples,)
#             fnn_loss = pinball_loss(
#                 y_true=actual,
#                 y_pred=fnn_pred,
#                 quantile=quantile
#             )
#             tick_losses['FNN'][horizon][quantile] = np.mean(fnn_loss)
        
#         # LQR tick loss: get predictions for quantile q, horizon h
#         if lqr_predictions is not None:
#             lqr_pred = lqr_predictions[:, q, h]  # (samples,)
#             lqr_loss = pinball_loss(
#                 y_true=actual,
#                 y_pred=lqr_pred,
#                 quantile=quantile
#             )
#             tick_losses['LQR'][horizon][quantile] = np.mean(lqr_loss)
        
    

# # === 2. Create Tick Loss Comparison Table ===
# print("\n2. Creating tick loss comparison table...")

# # Prepare data for comparison table
# comparison_data = []
# for horizon in config['data']['horizons']:
#     for quantile in config['data']['quantiles']:
#         fnn_loss = tick_losses['FNN'][horizon][quantile]
#         lqr_loss = tick_losses['LQR'][horizon][quantile]
#         improvement = ((lqr_loss - fnn_loss) / lqr_loss) * 100  # % improvement of FNN over LQR
#         print(fnn_loss, lqr_loss, improvement)
#         comparison_data.append({
#             'Horizon': horizon,
#             'Quantile': quantile,
#             'FNN_Loss': fnn_loss,
#             'LQR_Loss': lqr_loss,
#             'Improvement_%': improvement,
#             'Better_Model': 'FNN' if fnn_loss < lqr_loss else 'LQR'
#         })

# comparison_df = pd.DataFrame(comparison_data)
# print("\nTick Loss Comparison Table:")
# display(comparison_df.round(4))

# # === 3. Visualization: Tick Loss Comparison ===
# print("\n3. Creating tick loss comparison plots...")

# # Plot 1: Tick Loss by Quantile and Horizon
# fig, axes = plt.subplots(1, len(config['data']['horizons']), figsize=(5*len(config['data']['horizons']), 6))
# if len(config['data']['horizons']) == 1:
#     axes = [axes]

# for i, horizon in enumerate(config['data']['horizons']):
#     ax = axes[i]
    
#     quantiles = config['data']['quantiles']
#     fnn_losses = [tick_losses['FNN'][horizon][q] for q in quantiles]
#     lqr_losses = [tick_losses['LQR'][horizon][q] for q in quantiles]
    
#     x = np.arange(len(quantiles))
#     width = 0.35
    
#     bars1 = ax.bar(x - width/2, fnn_losses, width, label='Factor NN', alpha=0.8, color='skyblue')
#     bars2 = ax.bar(x + width/2, lqr_losses, width, label='LQR', alpha=0.8, color='lightcoral')
    
#     ax.set_xlabel('Quantiles')
#     ax.set_ylabel('Tick Loss')
#     ax.set_title(f'Tick Loss Comparison - Horizon {horizon}')
#     ax.set_xticks(x)
#     ax.set_xticklabels([f'{q:.2f}' for q in quantiles])
#     ax.legend()
#     ax.grid(True, alpha=0.3)
    
#     # Add value labels on bars
#     for bar in bars1:
#         height = bar.get_height()
#         ax.text(bar.get_x() + bar.get_width()/2., height,
#                 f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
#     for bar in bars2:
#         height = bar.get_height()
#         ax.text(bar.get_x() + bar.get_width()/2., height,
#                 f'{height:.3f}', ha='center', va='bottom', fontsize=9)

# plt.tight_layout()
# plt.show()

# #
# # === 5. Summary Statistics ===
# print("\n5. Summary statistics...")

# overall_stats = {
#     'FNN_avg_loss': np.mean([loss for horizon_dict in tick_losses['FNN'].values() 
#                             for loss in horizon_dict.values()]),
#     'LQR_avg_loss': np.mean([loss for horizon_dict in tick_losses['LQR'].values() 
#                             for loss in horizon_dict.values()]),
#     'FNN_wins': sum(1 for _, row in comparison_df.iterrows() if row['Better_Model'] == 'FNN'),
#     'LQR_wins': sum(1 for _, row in comparison_df.iterrows() if row['Better_Model'] == 'LQR'),
#     'avg_improvement': comparison_df['Improvement_%'].mean()
# }

# print(f"\nOverall Performance Summary:")
# print(f"  Average FNN Loss: {overall_stats['FNN_avg_loss']:.4f}")
# print(f"  Average LQR Loss: {overall_stats['LQR_avg_loss']:.4f}")
# print(f"  FNN wins: {overall_stats['FNN_wins']}/{len(comparison_df)} cases")
# print(f"  LQR wins: {overall_stats['LQR_wins']}/{len(comparison_df)} cases")
# print(f"  Average improvement: {overall_stats['avg_improvement']:.2f}%")

# print("\n✓ Tick loss evaluation completed!")


In [ ]:
# === 6. Forecast Visualization ===
print("\n" + "="*50)
print("FORECAST VISUALIZATION")
print("="*50)

# Plot forecast paths for each country and horizon
countries = list(scaled_test.keys())
n_countries_to_plot = len(countries)  # Plot first 4 countrie
# divide by 2 and round up for subplots
div_2 = (n_countries_to_plot + 1) // 2


# Calculate country boundaries for indexing into stacked arrays
country_boundaries = [0]
for df in test_data_list:
    country_boundaries.append(country_boundaries[-1] + len(df))

for h, horizon in enumerate(config['data']['horizons']):
    fig, axes = plt.subplots(div_2, 2, figsize=(15, 30))
    axes = axes.flatten()
    
    print(f"\nCreating forecast plots for horizon {horizon}...")
    
    for i, country in enumerate(countries[:n_countries_to_plot]):
        ax = axes[i]
        
        # Get country data boundaries
        start_idx = country_boundaries[i]
        end_idx = country_boundaries[i + 1]
        
        # Extract predictions and targets for this country and horizon
        # Targets: if 2D (samples, horizons), get column h; if 1D, use as-is
        if fnn_per_country_targets[country].ndim == 2:
            actual = fnn_per_country_targets[country][:, h]
        else:
            actual = fnn_per_country_targets[country]  # Single horizon case
        
        # Predictions: (samples, quantiles, horizons) -> get [:, :, h] for this horizon
        fnn_pred_this_horizon = fnn_per_country_predictions[country][:, h]  # (country_samples, quantiles)
        lqr_pred_this_horizon = lqr_per_country_predictions[country][:, h]  # (country_samples, quantiles)
        lqr_ar_pred_this_horizon = lqr_ar_per_country_predictions[country][:, h]  # (country_samples, quantiles)

        # Extract quantiles (assuming first is lower, second is upper)
        try:
            fnn_pred_q05 = fnn_pred_this_horizon[:, 0]  # Lower quantile
            fnn_pred_q95 = fnn_pred_this_horizon[:, 1]  # Upper quantile
            lqr_pred_q05 = lqr_pred_this_horizon[:, 0]
            lqr_pred_q95 = lqr_pred_this_horizon[:, 1]
            lqr_ar_pred_q05 = lqr_ar_pred_this_horizon[:, 0]
            lqr_ar_pred_q95 = lqr_ar_pred_this_horizon[:, 1]
        except IndexError:
            fnn_pred_q05 = [np.nan] * len(actual)
            fnn_pred_q95 = [np.nan] * len(actual)
            lqr_pred_q05 = [np.nan] * len(actual)
            lqr_pred_q95 = [np.nan] * len(actual)
            lqr_ar_pred_q05 = [np.nan] * len(actual)
            lqr_ar_pred_q95 = [np.nan] * len(actual)

            
        # Time index for plotting
        time_idx = range(len(actual))
        
        # Plot actual values
        ax.plot(time_idx, actual, 'k-', linewidth=2, label='Actual', alpha=0.8)
        
        # Plot FNN predictions with uncertainty band
        ax.fill_between(time_idx, fnn_pred_q05, fnn_pred_q95, 
                       alpha=0.3, color='blue', label='FNN 90% CI')
        ax.plot(time_idx, fnn_pred_this_horizon, 'b--', 
               linewidth=1.5, label='FNN Median', alpha=0.8)
        
        # Plot LQR predictions with uncertainty band  
        ax.fill_between(time_idx, lqr_pred_q05, lqr_pred_q95, 
                       alpha=0.2, color='red', label='LQR 90% CI')
        ax.plot(time_idx, lqr_pred_this_horizon, 'r:', 
               linewidth=1.5, label='LQR Median', alpha=0.8)
        
        # Plot LQR AR predictions with uncertainty band
        ax.fill_between(time_idx, lqr_ar_pred_q05, lqr_ar_pred_q95, 
                       alpha=0.2, color='green', label='LQR AR 90% CI')
        ax.plot(time_idx, lqr_ar_pred_this_horizon, 'g-.', 
               linewidth=1.5, label='LQR AR Median', alpha=0.8)
        
        ax.set_title(f'{country} - Horizon {horizon}')
        ax.set_xlabel('Time Period')
        ax.set_ylabel(config['data']['target'])
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)
    
    plt.suptitle(f'Forecast Comparison - {horizon}-Period Ahead', fontsize=16)
    plt.tight_layout()
    plt.show()

# # === 7. Quantile Coverage Analysis ===
# print("\n7. Quantile coverage analysis...")

# # Check if actual values fall within prediction intervals
# coverage_stats = {}

# for h, horizon in enumerate(config['data']['horizons']):
#     coverage_stats[horizon] = {}
    
#     # Get targets for this horizon
#     if fnn_targets.ndim == 2:
#         actual = fnn_targets[:, h]
#     else:
#         actual = fnn_targets
    
#     # For each model
#     for model_name, predictions in [('FNN', fnn_per_country_predictions), ('LQR', lqr_per_country_predictions)]:
#         # Get predictions for this horizon: (samples, quantiles)
#         preds_this_horizon = predictions[:, :, h]
        
#         coverage_stats[horizon][model_name] = {}
#         for q, quantile in enumerate(config['data']['quantiles']):
#             # Get lower and upper quantiles
#             pred = preds_this_horizon[:, q]
#             # Calculate coverage (percentage of actual values within prediction interval)
#             within_interval = (actual <= pred) 
#             coverage = np.mean(within_interval) * 100
            
#             # Expected coverage for q
#             expected_coverage = (quantile) * 100
            
#             coverage_stats[horizon][model_name][quantile] = {
#                 'coverage': coverage,
#                 'expected': expected_coverage,
#                 'difference': coverage - expected_coverage
#             }
            
#             print(f"Horizon {horizon} - {model_name}:")
#             print(f"  Actual coverage: {coverage:.1f}%")
#             print(f"  Expected coverage: {expected_coverage:.1f}%")
#             print(f"  Difference: {coverage - expected_coverage:+.1f}%")

#             # Plot coverage comparison for all quantiles 
#             fig, ax = plt.subplots(1, 1, figsize=(10, 6))

#             horizons = config['data']['horizons']
#             quantiles = config['data']['quantiles']

#             # Calculate positions for grouped bars
#             n_quantiles = len(quantiles)
#             n_horizons = len(horizons)
#             x = np.arange(n_horizons)
#             width = 0.35 / n_quantiles  # Adjust width based on number of quantiles

#     # Prepare data for all quantiles
#     for q_idx, quantile in enumerate(quantiles):
#         fnn_coverage = [coverage_stats[h]['FNN'][quantile]['coverage'] for h in horizons]
#         lqr_coverage = [coverage_stats[h]['LQR'][quantile]['coverage'] for h in horizons]
#         expected_coverage = coverage_stats[horizons[0]]['FNN'][quantile]['expected']  # Same for all horizons
        
#         # Calculate bar positions
#         fnn_pos = x - width/2 + q_idx * width * 2
#         lqr_pos = x + width/2 + q_idx * width * 2
        
#         # Plot bars for this quantile
#         bars1 = ax.bar(fnn_pos, fnn_coverage, width, 
#                         label=f'FNN Q{quantile:.2f}', alpha=0.8)
#         bars2 = ax.bar(lqr_pos, lqr_coverage, width, 
#                         label=f'LQR Q{quantile:.2f}', alpha=0.8)
        
#         # Add expected coverage line for this quantile
#         ax.axhline(y=expected_coverage, color='gray', linestyle='--', 
#                     alpha=0.7, linewidth=1)
        
#         # Add value labels on bars
#         for bars in [bars1, bars2]:
#             for bar in bars:
#                 height = bar.get_height()
#                 ax.text(bar.get_x() + bar.get_width()/2., height,
#                         f'{height:.1f}%', ha='center', va='bottom', fontsize=8)

#     ax.set_xlabel('Forecast Horizon')
#     ax.set_ylabel('Coverage (%)')
#     ax.set_title('Prediction Interval Coverage Analysis by Quantile')
#     ax.set_xticks(x)
#     ax.set_xticklabels([f'H{h}' for h in horizons])
#     ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#     ax.grid(True, alpha=0.3)

#     plt.tight_layout()
#     plt.show()

#     print("\n✓ Forecast visualization completed!")

In [ ]:
# === 8. Statistical Significance Testing ===
print("\n" + "="*50)
print("STATISTICAL SIGNIFICANCE TESTING")
print("="*50)

# Generic function for comparing two models
def compare_models_dm_test(model1_predictions, model2_predictions, targets, 
                          model1_name="Model1", model2_name="Model2", countries=None):
    """
    Perform Diebold-Mariano test for forecast accuracy comparison between any two models.
    
    Parameters:
    -----------
    model1_predictions : dict
        Dictionary with country keys and prediction arrays
    model2_predictions : dict  
        Dictionary with country keys and prediction arrays
    targets : dict
        Dictionary with country keys and target arrays
    model1_name : str
        Name of first model for display
    model2_name : str
        Name of second model for display
    countries : list
        List of countries to compare (if None, use all available)
    
    Returns:
    --------
    dict : Results of DM tests per country and aggregate
    """
    from scipy import stats
    
    if countries is None:
        countries = list(model1_predictions.keys())
    
    dm_results_per_country = {}
    dm_results_aggregate = {}

    # Per-Country Diebold-Mariano Tests
    for country in countries:
        if (model1_predictions.get(country) is not None and 
            model2_predictions.get(country) is not None):
            
            dm_results_per_country[country] = {}
            print(f"\n{country} - Diebold-Mariano Tests ({model1_name} vs {model2_name}):")
            
            for h, horizon in enumerate(config['data']['horizons']):
                dm_results_per_country[country][horizon] = {}
                
                for q, quantile in enumerate(config['data']['quantiles']):
                    # Get targets for this country and horizon
                    if targets[country].ndim == 2:
                        actual = targets[country][:, h]
                    else:
                        actual = targets[country]
                    
                    # Get predictions for this country, horizon, and quantile
                    model1_pred = model1_predictions[country][:, h][:, q]
                    model2_pred = model2_predictions[country][:, h][:, q]
                    
                    if len(actual) > 5:  # Need minimum samples for meaningful test
                        # Calculate quantile losses (more appropriate for quantile forecasting)
                        model1_qloss = pinball_loss(actual, model1_pred, quantile)
                        model2_qloss = pinball_loss(actual, model2_pred, quantile)
                        
                        # Diebold-Mariano test statistic
                        # H0: Both forecasts have equal accuracy
                        # H1: Forecasts have different accuracy
                        loss_diff = model1_qloss - model2_qloss  # Positive means Model1 worse
                        
                        # Calculate test statistic
                        mean_diff = np.mean(loss_diff)
                        var_diff = np.var(loss_diff, ddof=1)
                        
                        if var_diff > 0:
                            dm_stat = mean_diff / np.sqrt(var_diff / len(loss_diff))
                            p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))  # Two-tailed test
                            
                            dm_results_per_country[country][horizon][quantile] = {
                                'dm_statistic': dm_stat,
                                'p_value': p_value,
                                'significant': p_value < 0.1,
                                'better_model': model2_name if dm_stat > 0 else model1_name
                            }
                            
                            significance = "***" if p_value < 0.01 else "**" if p_value < 0.05 else "*" if p_value < 0.1 else ""
                            
                            print(f"  H{horizon} Q{quantile:.2f}: DM={dm_stat:.3f}, p={p_value:.3f}{significance} -> {dm_results_per_country[country][horizon][quantile]['better_model']}")
                        else:
                            dm_results_per_country[country][horizon][quantile] = {
                                'dm_statistic': 0,
                                'p_value': 1.0,
                                'significant': False,
                                'better_model': 'Tie'
                            }
                            print(f"  H{horizon} Q{quantile:.2f}: No variance in loss differences")
                    else:
                        dm_results_per_country[country][horizon][quantile] = {
                            'dm_statistic': np.nan,
                            'p_value': np.nan,
                            'significant': False,
                            'better_model': 'Insufficient data'
                        }
                        print(f"  H{horizon} Q{quantile:.2f}: Insufficient data for testing")

    return dm_results_per_country, dm_results_aggregate

# Use the generic function to compare FNN vs LQR
dm_results_per_country, dm_results_aggregate = compare_models_dm_test(
    fnn_per_country_predictions, 
    lqr_per_country_predictions, 
    fnn_per_country_targets,
    model1_name="FNN",
    model2_name="LQR",
    countries=countries
)

# Aggregate Diebold-Mariano Tests (for comparison)
def compute_aggregate_dm_tests(model1_predictions, model2_predictions, targets, 
                              model1_name="Model1", model2_name="Model2", countries=None):
    """
    Compute aggregate Diebold-Mariano tests across all countries for any two models.
    """
    from scipy import stats
    
    if countries is None:
        countries = list(model1_predictions.keys())
    
    dm_results_aggregate = {}
    
    print(f"\nAGGREGATE ANALYSIS - Diebold-Mariano Tests ({model1_name} vs {model2_name}):")
    for h, horizon in enumerate(config['data']['horizons']):
        dm_results_aggregate[horizon] = {}
        
        for q, quantile in enumerate(config['data']['quantiles']):
            # Aggregate data across all countries for this horizon and quantile
            all_model1_losses = []
            all_model2_losses = []
            
            for country in countries:
                if (model1_predictions.get(country) is not None and 
                    model2_predictions.get(country) is not None):
                    
                    # Get targets for this country and horizon
                    if targets[country].ndim == 2:
                        actual = targets[country][:, h]
                    else:
                        actual = targets[country]
                    
                    # Get predictions for this country, horizon, and quantile
                    model1_pred = model1_predictions[country][:, h][:, q]
                    model2_pred = model2_predictions[country][:, h][:, q]
                    
                    # Calculate quantile losses
                    model1_qloss = pinball_loss(actual, model1_pred, quantile)
                    model2_qloss = pinball_loss(actual, model2_pred, quantile)
                    
                    all_model1_losses.extend(model1_qloss)
                    all_model2_losses.extend(model2_qloss)
            
            if len(all_model1_losses) > 1:
                # Convert to numpy arrays
                all_model1_losses = np.array(all_model1_losses)
                all_model2_losses = np.array(all_model2_losses)
                
                # Diebold-Mariano test statistic
                loss_diff = all_model1_losses - all_model2_losses  # Positive means Model1 worse
                
                # Calculate test statistic
                mean_diff = np.mean(loss_diff)
                var_diff = np.var(loss_diff, ddof=1)
                
                if var_diff > 0:
                    dm_stat = mean_diff / np.sqrt(var_diff / len(loss_diff))
                    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))  # Two-tailed test
                    
                    dm_results_aggregate[horizon][quantile] = {
                        'dm_statistic': dm_stat,
                        'p_value': p_value,
                        'significant': p_value < 0.1,
                        'better_model': model2_name if dm_stat > 0 else model1_name
                    }
                    
                    significance = "***" if p_value < 0.01 else "**" if p_value < 0.05 else "*" if p_value < 0.1 else ""
                    
                    print(f"  H{horizon} Q{quantile:.2f}: DM={dm_stat:.3f}, p={p_value:.3f}{significance} -> {dm_results_aggregate[horizon][quantile]['better_model']}")
                else:
                    print(f"  H{horizon} Q{quantile:.2f}: No variance in loss differences")
            else:
                print(f"  H{horizon} Q{quantile:.2f}: Insufficient data for testing")
    
    return dm_results_aggregate

# Compute aggregate results for FNN vs LQR
dm_results_aggregate = compute_aggregate_dm_tests(
    fnn_per_country_predictions,
    lqr_per_country_predictions, 
    fnn_per_country_targets,
    model1_name="FNN",
    model2_name="LQR",
    countries=countries
)

# Create Diebold-Mariano summary table
print("\nDiebold-Mariano Test Results Summary:")
dm_summary_data = []

for country in countries:
    if country in dm_results_per_country:
        for horizon in config['data']['horizons']:
            for quantile in config['data']['quantiles']:
                dm_result = dm_results_per_country[country][horizon][quantile]
                dm_summary_data.append({
                    'Country': country,
                    'Horizon': horizon,
                    'Quantile': quantile,
                    'DM_Statistic': dm_result['dm_statistic'],
                    'P_Value': dm_result['p_value'],
                    'Significant': dm_result['significant'],
                    'Better_Model': dm_result['better_model']
                })

dm_summary_df = pd.DataFrame(dm_summary_data)
display(dm_summary_df.round(4))

# === 9. Per-Country Quantile Loss Analysis ===
print("\n9. Computing per-country Quantile Loss metrics...")

# Generic function for per-country metrics comparison
def compute_per_country_metrics(model1_predictions, model2_predictions, targets,
                               model1_name="Model1", model2_name="Model2", countries=None):
    """
    Calculate Quantile Loss for each country, quantile, and horizon for any two models.
    
    Parameters:
    -----------
    model1_predictions : dict
        Dictionary with country keys and prediction arrays
    model2_predictions : dict  
        Dictionary with country keys and prediction arrays
    targets : dict
        Dictionary with country keys and target arrays
    model1_name : str
        Name of first model for display
    model2_name : str
        Name of second model for display
    countries : list
        List of countries to analyze (if None, use all available)
    
    Returns:
    --------
    dict : Per-country metrics for each model
    """
    if countries is None:
        countries = list(model1_predictions.keys())
    
    per_country_metrics = {}

    for country in countries:
        if (model1_predictions.get(country) is not None and 
            model2_predictions.get(country) is not None):
            
            per_country_metrics[country] = {}
            
            for h, horizon in enumerate(config['data']['horizons']):
                per_country_metrics[country][horizon] = {}
                
                # Get targets for this country and horizon
                if targets[country].ndim == 2:
                    actual = targets[country][:, h]
                else:
                    actual = targets[country]
                
                for q, quantile in enumerate(config['data']['quantiles']):
                    per_country_metrics[country][horizon][quantile] = {}
                    
                    # Get predictions for this country, horizon, and quantile
                    model1_pred = model1_predictions[country][:, h][:, q]
                    model2_pred = model2_predictions[country][:, h][:, q]
                    
                    # Calculate Quantile Loss for each model
                    for model_name, pred in [(model1_name, model1_pred), (model2_name, model2_pred)]:
                        q_loss = np.mean(pinball_loss(actual, pred, quantile))
                        per_country_metrics[country][horizon][quantile][model_name] = q_loss
    
    return per_country_metrics

# Calculate metrics for FNN vs LQR
per_country_metrics = compute_per_country_metrics(
    fnn_per_country_predictions,
    lqr_per_country_predictions,
    fnn_per_country_targets,
    model1_name="FNN",
    model2_name="LQR", 
    countries=countries
)

# Create per-country metrics table and aggregate analysis
def create_metrics_comparison_table(per_country_metrics, countries, 
                                   model1_name="Model1", model2_name="Model2"):
    """
    Create comparison tables for any two models and compute aggregate metrics.
    """
    # Create per-country metrics table
    print(f"\nPer-Country Quantile Loss Performance ({model1_name} vs {model2_name}):")
    country_metrics_data = []

    for country in countries:
        if country in per_country_metrics:
            for horizon in config['data']['horizons']:
                for quantile in config['data']['quantiles']:
                    model1_qloss = per_country_metrics[country][horizon][quantile][model1_name]
                    model2_qloss = per_country_metrics[country][horizon][quantile][model2_name]
                    improvement = ((model2_qloss - model1_qloss) / model2_qloss) * 100
                    
                    country_metrics_data.append({
                        'Country': country,
                        'Horizon': horizon,
                        'Quantile': quantile,
                        f'{model1_name}_QLoss': model1_qloss,
                        f'{model2_name}_QLoss': model2_qloss,
                        'Improvement_%': improvement,
                        'Better_Model': model1_name if model1_qloss < model2_qloss else model2_name
                    })

    country_metrics_df = pd.DataFrame(country_metrics_data)
    display(country_metrics_df.round(4))

    # === Aggregate Summary ===
    print(f"\nAggregate Quantile Loss Summary ({model1_name} vs {model2_name}):")
    aggregate_metrics_data = []

    for horizon in config['data']['horizons']:
        for quantile in config['data']['quantiles']:
            # Aggregate across all countries
            all_model1_qloss = []
            all_model2_qloss = []
            
            for country in countries:
                if country in per_country_metrics:
                    all_model1_qloss.append(per_country_metrics[country][horizon][quantile][model1_name])
                    all_model2_qloss.append(per_country_metrics[country][horizon][quantile][model2_name])
            
            avg_model1_qloss = np.mean(all_model1_qloss)
            avg_model2_qloss = np.mean(all_model2_qloss)
            improvement = ((avg_model2_qloss - avg_model1_qloss) / avg_model2_qloss) * 100
            
            aggregate_metrics_data.append({
                'Horizon': horizon,
                'Quantile': quantile,
                f'{model1_name}_QLoss_Avg': avg_model1_qloss,
                f'{model2_name}_QLoss_Avg': avg_model2_qloss,
                'Improvement_%': improvement,
                'Better_Model': model1_name if avg_model1_qloss < avg_model2_qloss else model2_name
            })

    aggregate_df = pd.DataFrame(aggregate_metrics_data)
    print("\nAggregate Performance Across All Countries:")
    display(aggregate_df.round(4))
    
    return country_metrics_df, aggregate_df

# Generate tables for FNN vs LQR comparison
country_metrics_df, aggregate_df = create_metrics_comparison_table(
    per_country_metrics, countries, 
    model1_name="FNN", model2_name="LQR"
)

# === 10. Final Summary and Recommendations ===
print("\n" + "="*60)
print("FINAL QUANTILE LOSS EVALUATION SUMMARY")
print("="*60)

def generate_final_summary(country_metrics_df, per_country_metrics, countries,
                          model1_name="Model1", model2_name="Model2"):
    """
    Generate final summary and recommendations for any two models comparison.
    """
    # Calculate win rates for Quantile Loss only
    qloss_win_summary = {
        f'{model1_name}_wins': 0,
        f'{model2_name}_wins': 0,
        'total_comparisons': len(country_metrics_df)
    }

    for _, row in country_metrics_df.iterrows():
        # Quantile Loss comparison
        if row[f'{model1_name}_QLoss'] < row[f'{model2_name}_QLoss']:
            qloss_win_summary[f'{model1_name}_wins'] += 1
        else:
            qloss_win_summary[f'{model2_name}_wins'] += 1

    print(f"\nQuantile Loss Win Rate Summary ({model1_name} vs {model2_name}):")
    print("-" * 50)
    model1_wins = qloss_win_summary[f'{model1_name}_wins']
    total = qloss_win_summary['total_comparisons']
    model1_rate = (model1_wins / total) * 100

    print(f"Quantile Loss: {model1_name} wins {model1_wins}/{total} ({model1_rate:.1f}%)")

    # Country-by-Country Summary
    print(f"\nCountry-by-Country Performance Summary ({model1_name} vs {model2_name}):")
    print("-" * 60)
    for country in countries:
        if country in per_country_metrics:
            country_data = country_metrics_df[country_metrics_df['Country'] == country]
            country_model1_wins = sum(1 for _, row in country_data.iterrows() if row['Better_Model'] == model1_name)
            country_total = len(country_data)
            country_model1_rate = (country_model1_wins / country_total) * 100 if country_total > 0 else 0
            
            avg_improvement = country_data['Improvement_%'].mean()
            
            print(f"{country}: {model1_name} wins {country_model1_wins}/{country_total} ({country_model1_rate:.1f}%) | "
                  f"Avg improvement: {avg_improvement:+.2f}%")

    # Overall recommendation
    if model1_rate > 60:
        recommendation = f"{model1_name} shows superior quantile forecasting performance"
    elif model1_rate < 40:
        recommendation = f"{model2_name} shows superior quantile forecasting performance"
    else:
        recommendation = f"Both {model1_name} and {model2_name} show comparable quantile forecasting performance"

    print(f"\nOverall Recommendation: {recommendation}")

    # Best and worst performing quantiles and horizons
    print(f"\nPerformance by Quantile and Horizon ({model1_name} vs {model2_name}):")
    print("-" * 50)
    for horizon in config['data']['horizons']:
        print(f"\nHorizon {horizon}:")
        for quantile in config['data']['quantiles']:
            horizon_quantile_data = aggregate_df[
                (aggregate_df['Horizon'] == horizon) & 
                (aggregate_df['Quantile'] == quantile)
            ]
            if not horizon_quantile_data.empty:
                row = horizon_quantile_data.iloc[0]
                print(f"  Quantile {quantile:.2f}: {row['Better_Model']} wins | "
                      f"Improvement: {row['Improvement_%']:+.2f}%")

    print(f"\nKey Insights:")
    print("• Quantile Loss is the most relevant metric for quantile forecasting evaluation")
    print("• Per-country analysis reveals model performance heterogeneity across countries")
    print("• Different quantiles may have varying predictability across forecast horizons")
    print("• Country-specific characteristics may favor different modeling approaches")

    print(f"\n✓ Complete per-country quantile loss evaluation finished ({model1_name} vs {model2_name})!")

# Generate summary for FNN vs LQR
generate_final_summary(country_metrics_df, per_country_metrics, countries,
                      model1_name="FNN", model2_name="LQR")